# Azure OpenAI Function Calling Tutorial for Beginners

Welcome to this beginner-friendly tutorial on **Function Calling** with **Azure OpenAI**! Function Calling allows you to connect large language models (LLMs) like GPT models to external tools, APIs, or functions. The model can decide when to call a function and provide the necessary arguments based on the user's query.

This tutorial uses a Jupyter Notebook (`.ipynb`) format, so you can run the code cells interactively. We'll cover:

1. **Setup**: Prerequisites and environment configuration.
2. **Basics**: Defining functions and making your first call.
3. **Examples**: Real-world scenarios like weather lookup or math calculations.
4. **Best Practices**: Tips for production use.

**Note**: This tutorial assumes basic Python knowledge. We'll use the `openai` Python library to interact with Azure OpenAI.

## Prerequisites

- An active **Azure subscription**.
- An **Azure OpenAI resource** deployed with a GPT model (e.g., `gpt-4o` or `gpt-35-turbo`).
- Your **API key**, **endpoint**, and **deployment name** from the Azure portal.
- Python 3.8+ installed.

If you haven't set up Azure OpenAI yet:
1. Go to the [Azure Portal](https://portal.azure.com).
2. Create a new OpenAI resource.
3. Deploy a model under "Model deployments".
4. Note down your endpoint (e.g., `https://your-resource.openai.azure.com/`), API key, and deployment name.

## 1. Setup

First, install the required package and set up your environment variables.

**Run the cell below to install the OpenAI library.**

In [ ]:
!pip install openai

Now, import the library and configure your Azure OpenAI client. **Replace the placeholders with your actual values**.

For security, use environment variables in production (e.g., via `os.getenv`).

In [ ]:
from openai import AzureOpenAI
import os

# Replace with your values
AZURE_OPENAI_API_KEY = "your-api-key-here"
AZURE_OPENAI_ENDPOINT = "https://your-resource.openai.azure.com/"
DEPLOYMENT_NAME = "your-deployment-name"  # e.g., gpt-4o

# Initialize the client
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-02-15-preview",  # Use a recent API version supporting function calling
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

print("Client initialized successfully!")

## 2. Basics of Function Calling

Function Calling works in three steps:

1. **Define functions**: Describe what the function does, its parameters (in JSON schema format).
2. **Send to the model**: Include the functions in your chat completion request.
3. **Handle response**: If the model wants to call a function, it returns the function name and arguments. You execute it and feed the result back to the model.

Let's define a simple function: `get_current_weather` that takes a location and unit (e.g., Celsius or Fahrenheit).

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The unit to use"
                }
            },
            "required": ["location"]
        }
    }
]

Now, make a chat completion request. The model will decide if it needs to call the function.

In [ ]:
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]

response = client.chat.completions.create(
    model=DEPLOYMENT_NAME,
    messages=messages,
    functions=functions,
    function_call="auto"  # Let the model decide
)

response_message = response.choices[0].message
print(response_message)

If the model calls a function, the response will have `function_call` populated. Let's check:

In [ ]:
if response_message.function_call:
    print("Function called:", response_message.function_call.name)
    print("Arguments:", response_message.function_call.arguments)
else:
    print("No function call needed.")
    print("Content:", response_message.content)

Great! The model likely called `get_current_weather` with arguments like `{"location": "Boston, MA"}`.

In a real app, you'd now **execute the function** (e.g., query a weather API) and append the result as a `function` role message back to the conversation.

In [ ]:
# Mock function execution (in reality, call an actual API)
def get_current_weather(location, unit="fahrenheit"):
    # Simulated response
    if "boston" in location.lower():
        return {
            "temperature": 72,
            "unit": unit,
            "description": "Sunny"
        }
    return {"error": "Location not supported"}

# Parse arguments (JSON string to dict)
import json
function_args = json.loads(response_message.function_call.arguments)
function_response = get_current_weather(**function_args)

# Add to messages
messages.append(response_message)  # Add the assistant's function call message
messages.append(
    {
        "role": "function",
        "name": response_message.function_call.name,
        "content": json.dumps(function_response)
    }
)

# Get final response from model
second_response = client.chat.completions.create(
    model=DEPLOYMENT_NAME,
    messages=messages
)

print(second_response.choices[0].message.content)

The model now uses the function result to generate a natural response, e.g., "It's sunny in Boston with 72°F!"

## 3. Examples

### Example 1: Math Calculator

Let's add a calculator function for arithmetic operations.

In [ ]:
calculator_function = {
    "name": "calculator",
    "description": "Perform basic math calculations",
    "parameters": {
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "The math expression, e.g., '2 + 2 * 3'"
            }
        },
        "required": ["expression"]
    }
}

all_functions = [functions[0], calculator_function]  # Combine with weather function

In [ ]:
# Reset messages
messages = [{"role": "user", "content": "What is 15 * 4 + 7?"}]

response = client.chat.completions.create(
    model=DEPLOYMENT_NAME,
    messages=messages,
    functions=all_functions,
    function_call="auto"
)

response_message = response.choices[0].message

if response_message.function_call:
    function_args = json.loads(response_message.function_call.arguments)
    # Mock calculator (use eval for simplicity; use safer parsers in production)
    try:
        result = eval(function_args["expression"])
        function_response = {"result": result}
    except:
        function_response = {"error": "Invalid expression"}
    
    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": response_message.function_call.name,
            "content": json.dumps(function_response)
        }
    )
    
    final_response = client.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages
    )
    print(final_response.choices[0].message.content)
else:
    print(response_message.content)

### Example 2: Multiple Functions

The model can choose from multiple functions. Try querying: "Calculate 10 + 5 and tell me the weather in New York."

In [ ]:
messages = [{"role": "user", "content": "Calculate 10 + 5 and tell me the weather in New York."}]

response = client.chat.completions.create(
    model=DEPLOYMENT_NAME,
    messages=messages,
    functions=all_functions,
    function_call="auto"
)

print("Initial response:", response.choices[0].message)

For multiple calls, you may need to loop: check for function calls, execute, append, and repeat until no more calls.

## 4. Best Practices

- **Security**: Validate and sanitize function arguments before execution.
- **Error Handling**: Always handle cases where the model doesn't call a function or provides invalid args.
- **Descriptions**: Write clear function descriptions and parameter schemas—the model relies on these.
- **Rate Limits**: Respect Azure OpenAI quotas.
- **Production**: Use structured outputs (newer feature) for more reliable JSON parsing.
- **Testing**: Start with `function_call={"name": "your_function"}` to force a call for testing.

## Next Steps

- Explore [Azure OpenAI docs](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling).
- Integrate with real APIs (e.g., weather via OpenWeatherMap).
- Try parallel function calling for efficiency.

Thanks for following along! Questions? Experiment with the code cells.